In [1]:
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu" 
# if you want to use your TPU (note that torch_xla is not by default installed, install using pip install devinterp[tpu]) 
# import os 
# os.environ["USE_TPU_BACKEND"] = "1" 
# import torch_xla.core.xla_model as xm 
# DEVICE = xm.xla_device()

import warnings

import torch
import torchvision
from torch.nn import functional as F
from transformers import AutoModelForImageClassification

from devinterp.optim import SGLD
from devinterp.slt.sampler import estimate_learning_coeff_with_summary
from devinterp.utils import plot_trace, default_nbeta

warnings.filterwarnings("ignore")


/home/wisp/anaconda3/envs/llc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from rasp_models.peak import get_peak_model
from tracr.haiku_to_pytorch import haiku_to_pytorch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

bos = "BOS"
init_model = get_peak_model()
base_model = haiku_to_pytorch(init_model)

# Get vocab from the encoder's encoding_map
token_to_idx = init_model.input_encoder.encoding_map
vocab_list = list(token_to_idx.keys())
vocab_size = init_model.input_encoder.vocab_size
d_model = base_model.layers[0].attn.d_model

# Get actual max position from the Haiku model
max_pos = init_model.params['pos_embed']['embeddings'].shape[0]

print(f"Vocab size: {vocab_size}")
print(f"d_model: {d_model}")
print(f"Max positions: {max_pos}")

# Wrapper that adds proper embeddings
class TracrWithEmbeddings(nn.Module):
    def __init__(self, base_model, vocab_size, d_model, max_pos, token_to_idx, haiku_model):
        super().__init__()
        self.base_model = base_model
        self.token_to_idx = token_to_idx
        self.haiku_model = haiku_model
        
        # Create learnable embedding layers with correct sizes
        self.token_embed = nn.Embedding(vocab_size, d_model, dtype=torch.float64)
        self.pos_embed = nn.Embedding(max_pos, d_model, dtype=torch.float64)
        
        # Initialize with Haiku weights
        with torch.no_grad():
            token_emb_weights = torch.tensor(
                np.array(haiku_model.params['token_embed']['embeddings']),
                dtype=torch.float64
            )
            self.token_embed.weight.copy_(token_emb_weights)
            
            pos_emb_weights = torch.tensor(
                np.array(haiku_model.params['pos_embed']['embeddings']),
                dtype=torch.float64
            )
            self.pos_embed.weight.copy_(pos_emb_weights)
    
    def forward(self, x):
        # x is list of token sequences: [['BOS', 'a', 'b'], ...]
        
        # Convert to indices
        batch_indices = []
        for seq in x:
            indices = [self.token_to_idx[token] for token in seq]
            batch_indices.append(indices)
        
        # Convert to tensor
        x_indices = torch.tensor(batch_indices, dtype=torch.long, device=self.token_embed.weight.device)
        B, T = x_indices.shape
        
        # Get embeddings (these have gradients!)
        tok_emb = self.token_embed(x_indices)
        pos_emb = self.pos_embed(torch.arange(T, device=x_indices.device))
        x = tok_emb + pos_emb
        
        # Apply transformer layers from base model
        for layer in self.base_model.layers:
            x = layer(x)
        
        return x

# Add classification head wrapper
class TracrClassifier(nn.Module):
    def __init__(self, transformer, d_model, num_classes=2):
        super().__init__()
        self.transformer = transformer
        self.classifier = nn.Linear(d_model, num_classes, dtype=torch.float64)
        
        # Initialize classifier with small weights to prevent instability
        with torch.no_grad():
            self.classifier.weight.normal_(0, 0.01)
            self.classifier.bias.zero_()
    
    def forward(self, x):
        hidden = self.transformer(x)  # (batch, seq_len, d_model)
        pooled = hidden[:, -1, :]  # Use last token (batch, d_model)
        logits = self.classifier(pooled)  # (batch, num_classes)
        return logits

# Create model with classification head
base_transformer = TracrWithEmbeddings(base_model, vocab_size, d_model, max_pos, token_to_idx, init_model)
model = TracrClassifier(base_transformer, d_model, num_classes=2)

# Enable gradients
for param in model.parameters():
    param.requires_grad = True

model = model.to(DEVICE)

# Dataset
class PeakDataset(torch.utils.data.Dataset):
    def __init__(self, vocab, seq_len, num_samples):
        self.vocab = vocab
        self.seq_len = seq_len
        self.num_samples = num_samples
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        seq = [np.random.choice(self.vocab) for _ in range(self.seq_len)]
        seq = ["BOS"] + seq

        peaks = ["BOS"]
        for i in range(1, len(seq)):
            if i == 1 or i == len(seq) - 1:
                if len(seq) == 2:
                    peaks.append(False)
                elif i == 1:
                    peaks.append(seq[i] > seq[i + 1])
                elif i == len(seq) - 1:
                    peaks.append(seq[i] > seq[i - 1])
            else:
                peaks.append(seq[i] > seq[i - 1] and seq[i] > seq[i + 1])

        # Label = 1 if there is any peak, else 0
        label = int(any(peaks[1:]))

        return seq, label

def custom_collate(batch):
    """Keep sequences as lists, batch only labels"""
    sequences, labels = zip(*batch)
    return list(sequences), torch.tensor(labels, dtype=torch.long)

# Create dataset and loader
vocab = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
data = PeakDataset(vocab=vocab, seq_len=5, num_samples=10000)
loader = torch.utils.data.DataLoader(data, batch_size=256, shuffle=True, collate_fn=custom_collate)

# Proper evaluate function with cross-entropy loss
def evaluate(model, batch):
    sequences, labels = batch  
    labels = labels.to(DEVICE)
    
    # Get logits from model
    logits = model(sequences)  # (batch, 2)
    
    # Check for NaN/inf in logits before computing loss
    if torch.isnan(logits).any() or torch.isinf(logits).any():
        print("NaN/Inf in logits!")
        print("Logits stats - min:", logits.min().item(), "max:", logits.max().item())
        return torch.tensor(float('inf'), device=DEVICE)

    # Compute proper cross-entropy loss
    loss = F.cross_entropy(logits, labels)
    
    # Debug NaN if it occurs
    if torch.isnan(loss):
        print("NaN in loss!")
        print("Logits:", logits[:5])
        print("Labels:", labels[:5])
    
    return loss

learning_coeff_stats = estimate_learning_coeff_with_summary(
    model,
    loader=loader,
    evaluate=evaluate,
    sampling_method=SGLD,
    optimizer_kwargs=dict(
        lr=1e-5,  # Much lower learning rate
        localization=100.0, 
        nbeta=default_nbeta(loader)
    ),
    num_chains=3,
    num_draws=100,
    num_burnin_steps=0,
    num_steps_bw_draws=1,
    device=DEVICE,
    online=True,
)

trace = learning_coeff_stats["loss/trace"]
print(learning_coeff_stats.keys())

plot_trace(
    trace,
    "Loss",
    x_axis="Step",
    title=f"Loss Trace, avg LLC = {sum(learning_coeff_stats['llc/means']) / len(learning_coeff_stats['llc/means']):.2f}",
    plot_mean=False,
    plot_std=False,
    fig_size=(12, 9),
    true_lc=None,
)

Vocab size: 12
d_model: 127
Max positions: 11
tensor([[ 0.0291,  0.0113],
        [ 0.0261, -0.0103],
        [ 0.0355, -0.0175],
        [ 0.0141, -0.0277],
        [ 0.0310, -0.0076],
        [ 0.0261, -0.0103],
        [ 0.0261, -0.0103],
        [ 0.0564, -0.0225],
        [ 0.0291,  0.0113],
        [ 0.0310, -0.0076],
        [ 0.0155, -0.0173],
        [ 0.0261, -0.0103],
        [ 0.0355, -0.0175],
        [ 0.0274, -0.0074],
        [ 0.0355, -0.0175],
        [ 0.0274, -0.0074],
        [ 0.0271,  0.0116],
        [ 0.0271,  0.0116],
        [ 0.0271,  0.0116],
        [ 0.0141, -0.0277],
        [ 0.0355, -0.0175],
        [ 0.0141, -0.0277],
        [ 0.0141, -0.0277],
        [ 0.0141, -0.0277],
        [ 0.0291,  0.0113],
        [ 0.0274, -0.0074],
        [ 0.0564, -0.0225],
        [ 0.0141, -0.0277],
        [ 0.0271,  0.0116],
        [ 0.0355, -0.0175],
        [ 0.0274, -0.0074],
        [ 0.0355, -0.0175],
        [ 0.0355, -0.0175],
        [ 0.0155, -0.0173],
  

Chain 0:   1%|          | 1/100 [00:00<00:13,  7.10it/s]

tensor([[ 0.0141, -0.0277],
        [ 0.0271,  0.0116],
        [ 0.0141, -0.0277],
        [ 0.0355, -0.0175],
        [ 0.0271,  0.0116],
        [ 0.0564, -0.0225],
        [ 0.0271,  0.0116],
        [ 0.0355, -0.0175],
        [ 0.0355, -0.0175],
        [ 0.0564, -0.0225],
        [ 0.0043, -0.0033],
        [ 0.0043, -0.0033],
        [ 0.0141, -0.0277],
        [ 0.0274, -0.0074],
        [ 0.0271,  0.0116],
        [ 0.0271,  0.0116],
        [ 0.0141, -0.0277],
        [ 0.0274, -0.0074],
        [ 0.0274, -0.0074],
        [ 0.0274, -0.0074],
        [ 0.0291,  0.0113],
        [ 0.0141, -0.0277],
        [ 0.0310, -0.0076],
        [ 0.0310, -0.0076],
        [ 0.0564, -0.0225],
        [ 0.0291,  0.0113],
        [ 0.0274, -0.0074],
        [ 0.0310, -0.0076],
        [ 0.0274, -0.0074],
        [ 0.0271,  0.0116],
        [ 0.0564, -0.0225],
        [ 0.0355, -0.0175],
        [ 0.0271,  0.0116],
        [ 0.0141, -0.0277],
        [ 0.0271,  0.0116],
        [ 0.0271,  0

Chain 0:   2%|▏         | 2/100 [00:00<00:13,  7.02it/s]

tensor([[ 31.1481, -19.3501],
        [ 31.5460, -19.5945],
        [ 31.7162, -19.6956],
        [ 31.7143, -19.6942],
        [ 31.5259, -19.5893],
        [ 31.9837, -19.8645],
        [ 31.9320, -19.8252],
        [ 31.7212, -19.6734],
        [ 31.5554, -19.6011],
        [ 31.8671, -19.8066],
        [ 31.9366, -19.8277],
        [ 31.5378, -19.5967],
        [ 31.1481, -19.3514],
        [ 31.7128, -19.6610],
        [ 31.8721, -19.8104],
        [ 31.7132, -19.6936],
        [ 31.7118, -19.6928],
        [ 31.5348, -19.5948],
        [ 31.8707, -19.8093],
        [ 31.7158, -19.6952],
        [ 31.9947, -19.8718],
        [ 31.7085, -19.6571],
        [ 31.7058, -19.6888],
        [ 31.9363, -19.8283],
        [ 31.8646, -19.8052],
        [ 31.7118, -19.6930],
        [ 31.9362, -19.8280],
        [ 31.9999, -19.8752],
        [ 31.1463, -19.3497],
        [ 31.9626, -19.8474],
        [ 31.9578, -19.8440],
        [ 31.7204, -19.6733],
        [ 31.9940, -19.8711],
        [ 

...

Chain 2:  15%|█▌        | 15/100 [00:02<00:11,  7.16it/s]

tensor([[-1.1644,  1.1655],
        [-1.0787,  1.0841],
        [-0.9929,  0.9789],
        [-0.9654,  0.9743],
        [-0.9197,  0.9202],
        [-0.9586,  0.9684],
        [-0.8882,  0.9563],
        [-0.9231,  0.9229],
        [-1.0219,  1.0080],
        [-0.9955,  1.0039],
        [-0.9707,  1.0360],
        [-1.0457,  1.0381],
        [-1.0637,  1.1270],
        [-0.9542,  1.0206],
        [-1.1170,  1.1317],
        [-1.0047,  0.9905],
        [-1.0123,  1.0766],
        [-1.0309,  1.0671],
        [-1.0653,  1.0574],
        [-0.8255,  0.8645],
        [-0.9317,  0.9248],
        [-0.9307,  0.9238],
        [-0.8647,  0.9325],
        [-1.0650,  1.0705],
        [-1.0169,  1.0329],
        [-1.0816,  1.0545],
        [-1.1091,  1.0693],
        [-1.0942,  1.0857],
        [-0.9078,  0.9555],
        [-1.0240,  1.0236],
        [-1.1193,  1.1651],
        [-0.8676,  0.8806],
        [-0.9125,  0.9509],
        [-0.9856,  0.9856],
        [-0.9440,  0.9304],
        [-1.1388,  1

Chain 2:  17%|█▋        | 17/100 [00:02<00:11,  7.25it/s]


tensor([[-0.8968,  0.8683],
        [-0.9010,  0.9012],
        [-0.9360,  0.9777],
        [-0.9963,  1.0173],
        [-0.9311,  0.9309],
        [-0.9447,  0.8867],
        [-0.9289,  0.9260],
        [-0.9773,  0.9475],
        [-0.9778,  0.9745],
        [-0.9331,  0.9743],
        [-0.9069,  0.8888],
        [-1.0367,  1.0749],
        [-0.9237,  0.9215],
        [-0.9192,  0.9608],
        [-0.8704,  0.8183],
        [-1.2188,  1.2556],
        [-1.0160,  0.9527],
        [-0.9852,  1.0518],
        [-0.8958,  0.9189],
        [-0.8330,  0.8333],
        [-0.8006,  0.7534],
        [-1.0547,  0.9870],
        [-0.9270,  0.9068],
        [-0.9289,  0.9110],
        [-0.9496,  0.9491],
        [-0.9561,  0.9554],
        [-0.9569,  0.9539],
        [-0.9518,  0.9496],
        [-0.8777,  0.8498],
        [-0.9717,  0.9935],
        [-1.0473,  0.9809],
        [-0.7576,  0.7837],
        [-1.5591,  1.5934],
        [-0.8608,  0.8625],
        [-0.8959,  0.8670],
        [-0.9796,  0

KeyboardInterrupt: 